In [1]:
!pip install torch-geometric


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.5 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv

In [3]:

# 1. Load the Cora dataset (a citation network graph)
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]  # The graph data

Processing...
Done!


In [4]:
# 2. Define the GCN model
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [5]:
# 3. Initialize the model, optimizer, and loss
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(dataset.num_node_features, 16, dataset.num_classes).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)


In [6]:
# 4. Training loop
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()


In [7]:
# 5. Testing function
def test():
    model.eval()
    logits = model(data)
    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs


In [8]:
# 6. Run training
for epoch in range(1, 201):
    loss = train()
    train_acc, val_acc, test_acc = test()
    if epoch % 20 == 0:
        print(f"Epoch {epoch:03d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}")

Epoch 020, Loss: 0.2869, Train Acc: 1.0000, Val Acc: 0.7740, Test Acc: 0.7980
Epoch 040, Loss: 0.0597, Train Acc: 1.0000, Val Acc: 0.7760, Test Acc: 0.7990
Epoch 060, Loss: 0.0496, Train Acc: 1.0000, Val Acc: 0.7800, Test Acc: 0.8100
Epoch 080, Loss: 0.0474, Train Acc: 1.0000, Val Acc: 0.7680, Test Acc: 0.7850
Epoch 100, Loss: 0.0376, Train Acc: 1.0000, Val Acc: 0.7640, Test Acc: 0.7920
Epoch 120, Loss: 0.0265, Train Acc: 1.0000, Val Acc: 0.7700, Test Acc: 0.7970
Epoch 140, Loss: 0.0366, Train Acc: 1.0000, Val Acc: 0.7760, Test Acc: 0.8040
Epoch 160, Loss: 0.0360, Train Acc: 1.0000, Val Acc: 0.7760, Test Acc: 0.8050
Epoch 180, Loss: 0.0265, Train Acc: 1.0000, Val Acc: 0.7720, Test Acc: 0.7970
Epoch 200, Loss: 0.0312, Train Acc: 1.0000, Val Acc: 0.7660, Test Acc: 0.8010
